# Weyl 群扩张群的泛性质 

In [13]:
pwd

'/home/rex/work_space/1 jupyter/0 项目/04 WeylGroupExtension'

In [2]:
cd work_space/1\ jupyter/0\ 项目/04\ WeylGroupExtension/

/home/rex/work_space/1 jupyter/0 项目/04 WeylGroupExtension


In [ ]:
cd Documents/jupyter/0-items/WeylGroupExtension/

In [14]:
load("weyl_group_extension.sage")

In [15]:
import time
from pprint import pprint
class Time():
    '''
    计时器
    输入：tic()开始
    输出：toc()结束
    误差：0.01s
    '''
    def tic(self):
        self.time = time.time()
    def toc(self,text='用时：'):
        t = self.time
        self.tic()
        print(text+'%.3f'%(self.time-t))
tt = Time()
tic = tt.tic
toc = tt.toc

目前已有函数
1. ExpOfNilpotentMat(mat, max_depth=36, base_ring=ZZ) # 返回幂零阵的指数矩阵
2. Thetas(s,n) 返回生成元集合 \{\theta_i\}_{i=1}^n
3. OrderMatrixOfGens(gens) 返回生成元的群阶矩阵
4. GroupByOrderMathix(order_mat) 群阶矩阵作为泛性质，生成群
5. GroupTreeOfMaxDepth(G,depth) 生成群树（BFS）
6. UniversalPropertyOfGroup(G,max_depth=-1) 求群生成元的泛性质
7. relations2elements(relations,gens) 生成元通过 gens 转为群的子集

## 知识背景

### 极小泛性质

1. 上边算法求生成元泛性质，给出的是相对粗糙的刻画；
2. 一般地，很难有直接构造的算法，求“最精细”的刻画；
3. 难处可以归结为：对任意自由群商群，判断表示式是否等于1
4. 一些特殊情形，比如 Coxeter 群，Humphreys 书给出了判断等于1的充要方法；
5. 但就一般情形来说，这很难做到的，以至于专业研究群论的工具 GAP 有时都判断不了

### 函数说明及相关性质

1. 函数说明：
   - GroupTreeOfMaxDepth(G,depth) 群的 0-depth 层
   - UniversalPropertyOfGroup(G,max_depth=-1) 返回值有三个（默认生成整树）：
      - tree 群 G 的 0-max_depth 层
      - tree_str 群 G 的 0-max_depth 层，元素用 reduced expression  
      - relations 字典，**能够生成 0-max_depth 层，且键值 <= max_depth 的泛性质**
2. 泛性质等价
   - relations 给出的是粗刻画，需进一步加细
   - relations 可以将 0-max_depth 所有的 expression 简约化
   - 反之，如果 0-max_depth 所有的 expression 简约化已知，可导出 relations
   - 等价关系：若 relaitons1 和 relations2 的键值均在 0-max_depth 层，且将 0-max_depth 层的所有 expression 简约化，则二者等价
   - 等价的两个泛性质关系可相互生成
3. 基于上边的讨论，从泛性质的粗刻画出发，逐步细化

### Weyl 群及其扩张群的相关结论

观察前几阶的测试结论，易得性质猜想:
1. 确定 Weyl 群的泛性质，需获取前 n 层：
   - A,D,E 型：前 3 层
   - B,C,F 型：前 4 层
   - G 型：前 6 层
2. 确定 Weyl 扩张群的泛性质，需要获取前 n 层：
   - A,D,E 型：前 4 层
   - B,C,F 型：前 4 层
   - G 型：前 6 层
3. 此外，二者泛性质可以只用包含两元素的情形

## A 型 Weyl 扩张群的泛性质

In [ ]:
fgens = FreeGroup(5).gens()
show_relations = lambda relations:table([(relation2element(a,fgens),relation2element(b,fgens)) for a,b in relations])
str2set = lambda r:set(r.replace("-",""))

In [33]:
# 初始化
s,n,k = "A",3,4
print(s,n)
gens = Thetas(s,n)
G = MatrixGroup(gens)
# 计时
tic()
tree = GroupTreeOfMaxDepth(G,k)
toc("获取群树用时")
record_len = [len(i) for i in tree] # 记录各层长度数据
print("各层数目",record_len)

# 测试
FG = FreeGroup(n)
gens = FG.gens()
type_1 = [(gens[i]^4,FG.one()) for i in range(n)]
type_2 = [(gens[i]*gens[i+1]*gens[i],gens[i+1]*gens[i]*gens[i+1]) for i in range(n-1)]
type_3 = [(gens[i]*gens[j],gens[j]*gens[i]) for i in range(n) for j in range(n) if i-j>1]
type_4 = [(gens[i]^2*gens[i+1]^2,gens[i+1]^2*gens[i]^2) for i in range(n-1)]
type_5 = [(gens[i]*gens[i+1]^2*gens[i],gens[i+1]^2) for i in range(n-1)]

elements = type_1 + type_2 + type_3 + type_5 + type_4
QG = FG / [a/b for a,b in elements] # 商群
tree = GroupTreeOfMaxDepth(QG,k)
tree_len = [len(i) for i in tree]
print("修改后的各层数目",tree_len,"\n是否相等",tree_len==record_len)


A 3
获取群树用时0.718
各层数目 [1, 3, 7, 15, 21]
修改后的各层数目 [1, 3, 8, 19, 34] 
是否相等 False


In [23]:
table(elements)

,
,
,
,


In [8]:
table(main("A",2))

A 2
获取群树用时0.045
各层数目 [1, 2, 4, 7, 7]
修改后的各层数目 [1, 2, 4, 7, 7] 
是否相等 True


,
,
,
,


In [9]:
table(main("A",3))

A 3
获取群树用时0.632
各层数目 [1, 3, 7, 15, 21]
修改后的各层数目 [1, 3, 8, 19, 34] 
是否相等 False


,
,
,
,
,
,
,
,


In [10]:
table(main("A",4))

A 4
获取群树用时12.255
各层数目 [1, 4, 13, 37, 86]
修改后的各层数目 [1, 4, 13, 37, 86] 
是否相等 True


,
,
,
,
,
,
,
,
,
,
,


### A2

In [ ]:
#### 修改 A2 情形
s,n,k = "A",2,4
gens = Thetas(s,n)
G = MatrixGroup(gens)

tic()
tree,relations = UniversalPropertyOfGroup(G,k)[1:]
toc()
layer_len = [len(i) for i in tree] # 记录各层长度数据
print("各层数目",layer_len)

## 初始 relations
### 去除单个幂次
# relations = [r for r in relations if len(str2set(r[0]))!=1]

m = len(relations)
print("生成关系个数",m)
show_relations(relations)

In [ ]:
pop = [3]
new_relations = [a for i,a in enumerate(relations) if i not in pop]
show_relations(new_relations)

In [ ]:
#### 测试区域 ####
FG = FreeGroup(n)
gens = FG.gens()
test_elements = relations2elements(new_relations,gens)
QG = FG/test_elements # 商群
tree = GroupTreeOfMaxDepth(QG,k)
tree_len = [len(i) for i in tree]
print("修改后的各层数目",tree_len,"\n是否相等",tree_len==layer_len)

### A3

In [ ]:
s,n,k = "A",3,4
gens = Thetas(s,n)
G = MatrixGroup(gens)

tic()
tree,relations = UniversalPropertyOfGroup(G,k)[1:]
toc()

layer_len = [len(i) for i in tree] # 记录各层长度数据
print("各层数目",layer_len)

## 初始 relations
### 去除单个幂次
relations = [r for r in relations if len(str2set(r[0]))!=1]


m = len(relations)
print("生成关系个数",m)
show_relations(relations)

In [ ]:
#### 修改 A3 情形
pop = [m-2,m-2,m-3] + [4,5]
new_relations = [a for i,a in enumerate(relations) if i not in pop]
show_relations(new_relations)

In [ ]:
#### 测试区域 ####
FG = FreeGroup(n)
gens = FG.gens()
test_elements = relations2elements(new_relations,gens)
QG = FG/test_elements # 商群
tree = GroupTreeOfMaxDepth(QG,k)
tree_len = [len(i) for i in tree]
print("修改后的各层数目",tree_len,"\n是否相等",tree_len==layer_len)

## 收纳

In [ ]:
# 弃用-极小性不可用
def MinimalUniversalProperty(G,max_depth=4):
    """
    1. 求群的极小泛性质
    2. max_depth 根据规律拟定
    """
    # 初始化
    tree,relations = UniversalPropertyOfGroup(G,max_depth)[1:]
    layer_len = [len(i) for i in tree] # 记录各层长度数据
    FG = FreeGroup(n) # 自由群
    elements = relations2elements(relations,FG.gens()) # 初始商集
    necessary = []
    # 逐步化归
    while len(elements):
        print("当前长度",len(elements),len(necessary))
        ele = elements.pop(-1)
        test_elements = elements + necessary
        QG = FG/test_elements # 商群
        tree = GroupTreeOfMaxDepth(QG,max_depth)
        if [len(i) for i in tree] != layer_len: # 个数增多 
            necessary.append(ele)
    necessary.reverse()
    return necessary